# IV.D - Sensitivity Analysis: Scene Appearance 

In [ ]:
# !pip install --upgrade scikit-image
# !pip install pyproj
# !pip install tqdm
# !pip install -U scipy==1.0.0

### Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import os, sys

from itertools import product
import matplotlib.image as mpimg
import pyproj

from skimage import transform

sys.path.append("../../datasets")
from dataLoader import DataLoader, preprocess_time

from tf.keras.models import Model, load_model
from tf.keras.layers import Input
from tf.keras.losses import mean_squared_error

transLabels = ["dirty", "daylight", "night", "sunrisesunset", "dawndusk", "sunny", "clouds", 
               "fog", "storm", "snow", "warm", "cold", "busy", "beautiful", "flowers", "spring", 
               "summer", "autumn", "winter", "glowing", "colorful", "dull", "rugged", "midday", 
               "dark", "bright", "dry", "moist", "windy", "rain", "ice", "cluttered", "soothing", 
               "stressful", "exciting", "sentimental", "mysterious", "boring", "gloomy", "lush"]


batchSize = 1
pathToModel = "../IV.B_ablation_study/denseNet/gr_oh_loc_time_TA/weights.30-0.57407.hdf5"
gpuNumber = 4

## GPU selection
import tensorflow as tf 
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[gpuNumber], 'GPU')
tf.config.experimental.set_memory_growth(gpus[gpuNumber], False)



#######################
##    Custom MSE     ##
#######################
# We will compute the MSE only for the consistent inputs
def transient_mse(y_true, y_pred):
    return tf.sum(mean_squared_error(y_true[0::2,:], y_pred[0::2,:]), axis=-1)



#######################
## Deprocess time
#######################
def deprocess_time(time):
    month, hour = time
    month = (11.0 / 2.0) * (month + 1) + 1
    hour = (23.0 / 2.0) * (hour + 1)
    return (int(round(month)), int(round(hour)))

def deprocess_loc(loc):
    _earth_radius = 6378137.0
    x,y,z = loc
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    
    lon, lat, alt = pyproj.transform(ecef, lla, x * _earth_radius, y*_earth_radius, z*_earth_radius, radians=False)
    return (lat, lon, alt)

Using TensorFlow backend.


### Load architecture and get pointers to specific layers
As we will process the features from multiple timestamps, we avoid re-processing the features for the ground-level image, location, and satellite image

In [ ]:
baseModel = load_model(pathToModel, custom_objects={"transient_mse": transient_mse})
print(baseModel.summary())

groundBranchModel = Model(baseModel.get_layer("groundInput").input, 
                          baseModel.get_layer("batch_normalization_2").output)
aerialBranchModel = Model(baseModel.get_layer("aerialInput").input, 
                          baseModel.get_layer("batch_normalization_4").output)
locBranchModel = Model(baseModel.get_layer("locationInput").input, 
                       baseModel.get_layer("batch_normalization_7").output)
timeBranchModel =  Model(baseModel.get_layer("timeInput").input, 
                         baseModel.get_layer("batch_normalization_10").output)


combinedFeaturesInput = Input(shape=(512,), name='concatenate_1_proxy')
combinedFeatures = baseModel.get_layer("consist_fc1")(combinedFeaturesInput)
combinedFeatures = baseModel.get_layer("batch_normalization_11")(combinedFeatures)
combinedFeatures = baseModel.get_layer("consist_fc2")(combinedFeatures)
combinedFeatures = baseModel.get_layer("batch_normalization_12")(combinedFeatures)
combinedFeatures = baseModel.get_layer("consist_fc3")(combinedFeatures)
combineModel = Model(combinedFeaturesInput, combinedFeatures)

--------------
--------------
--------------
--------------


### Selecting a few locations from the dataset to perform the analysis in SupMat-Figure 11

In [4]:
listOfIndex = [0, 3, 8, 12, 14, 21, 10, 2, 42, 28]
listOfTimeZone = [2, 1, -5, 1, -7, -8, 1, -7, 1, -3]
listOfLocations = [(37.98435301120001, 23.7326566772, -1.862645149230957e-09),
                  (45.46370000000002, 9.18814, -9.313225746154785e-10),
                  (33.749744744400004, -84.3883686381, 9.313225746154785e-10),
                  (50.1065184167, 8.678006826950002, 1.862645149230957e-09),
                  (32.904500000000006, -111.2479, 0.0),
                  (33.2648176, -116.38341830000002, 1.862645149230957e-09),
                  (42.2480919, 3.1227702, 9.313225746154785e-10),
                  (36.057900000000004, -112.1255, -9.313225746154785e-10),
                  (47.36450898660001, 7.1540129417300005, -1.862645149230957e-09),
                  (-30.034253501700007, -51.231207124200004, 9.313225746154785e-10)]

noonProbList = [[] for _ in range(24)]
eightAMProbList = [[] for _ in range(24)]
eightPMProbList = [[] for _ in range(24)]

janProbList = [[] for _ in range(12)]
julProbList = [[] for _ in range(12)]
novProbList = [[] for _ in range(12)]

### Predicting the consistency probability for all hours and months

In [6]:
dl = DataLoader("test", 
                includeLocation = True, 
                includeSatellite = True, 
                outputTransientAttributes = True)

pbar = tqdm(total=len(list(set(dl.aerialPaths))))

for batch, _ in dl.loadTestDataInBatches(batchSize, allTestSet=True):
    pbar.update(1)
    grImg, aeImg, locInfo, timeInfo = [batch[i][0:1] for i in range(len(batch))]
    
    dLoc = deprocess_loc(locInfo[0])
    dTime = deprocess_time(timeInfo[0])
    
    if dLoc in listOfLocations:
        idx = listOfLocations.index(dLoc)
        tz = listOfTimeZone[idx]
        
        grFeatures = groundBranchModel.predict_on_batch(grImg)
        aeFeatures = aerialBranchModel.predict_on_batch(aeImg)
        locFeatures = locBranchModel.predict_on_batch(locInfo)
        
        noonHour = (12 - tz) % 24
        tFeatures = timeBranchModel.predict_on_batch(preprocess_time((dTime[0], noonHour)).reshape(1,-1))
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, tFeatures))
        consistentProb = consistModel.predict_on_batch(concatFV)[0][0]
        noonProbList[(dTime[1] + tz) % 24] += [consistentProb]
        
        eightAMHour = (8 - tz) % 24
        tFeatures = timeBranchModel.predict_on_batch(preprocess_time((dTime[0], eightAMHour)).reshape(1,-1))
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, tFeatures))
        consistentProb = consistModel.predict_on_batch(concatFV)[0][0]
        eightAMProbList[(dTime[1] + tz) % 24] += [consistentProb]
        
        eightPMHour = (20 - tz) % 24
        tFeatures = timeBranchModel.predict_on_batch(preprocess_time((dTime[0], eightPMHour)).reshape(1,-1))
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, tFeatures))
        consistentProb = consistModel.predict_on_batch(concatFV)[0][0]
        eightPMProbList[(dTime[1] + tz) % 24] += [consistentProb]
        
        
        ### Months
        tFeatures = timeBranchModel.predict_on_batch(preprocess_time((1, dTime[1])).reshape(1,-1))
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, tFeatures))
        consistentProb = consistModel.predict_on_batch(concatFV)[0][0]
        janProbList[dTime[0] - 1] += [consistentProb]
        
        tFeatures = timeBranchModel.predict_on_batch(preprocess_time((7, dTime[1])).reshape(1,-1))
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, tFeatures))
        consistentProb = consistModel.predict_on_batch(concatFV)[0][0]
        julProbList[dTime[0] - 1] += [consistentProb]
        
        tFeatures = timeBranchModel.predict_on_batch(preprocess_time((11, dTime[1])).reshape(1,-1))
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, tFeatures))
        consistentProb = consistModel.predict_on_batch(concatFV)[0][0]
        novProbList[dTime[0] - 1] += [consistentProb]

pbar.close()

24931it [14:46, 28.13it/s]                           

[[0.0057944283, 0.00022353184, 0.08110286, 0.19983412, 0.08791109, 0.28759634, 0.12428803, 0.00017437116, 0.04386024, 0.0003202247, 0.00849421, 0.08047132, 0.005425268, 0.00051638624, 0.16164543, 0.0007776038, 0.0016126313, 0.06605151, 0.0047293683, 0.0006174054, 0.00096932985, 0.000353611, 0.001245826, 0.013951682, 0.03461376, 0.19505142, 0.018700086, 9.9505174e-05, 0.077832475, 0.00020175376, 0.0059534763, 0.005052252, 0.25479203, 0.014319412, 0.0011338098, 0.003731357], [0.0009601727, 0.035794046, 0.011561331, 0.081031926, 0.0032096745, 0.002229285, 0.08167198, 0.027910087, 0.003336688, 0.0027725615, 0.00023627252, 0.0032541426, 0.0009302066, 0.08014129, 0.01792974, 0.0054541947, 0.00074391835, 0.023298988, 0.017496897, 0.029347891, 0.009721024, 0.011212278, 0.010856131, 0.14328946, 0.01592303, 0.14085981, 0.00042402057, 0.026796274, 0.016156284, 0.28844887, 0.07373831, 0.00018134287, 0.005429721, 0.025969198, 0.023423413, 0.0011561678, 0.00014626412, 0.004457351, 3.740837e-05, 0.00

In [7]:
avgNoonProbList = [-1 if len(x) == 0 else np.mean(x) for x in noonProbList]
avgEightAMProbList = [-1 if len(x) == 0 else np.mean(x) for x in eightAMProbList]
avgEightPMProbList = [-1 if len(x) == 0 else np.mean(x) for x in eightPMProbList]

avgJanProbList = [-1 if len(x) == 0 else np.mean(x) for x in janProbList]
avgJulProbList = [-1 if len(x) == 0 else np.mean(x) for x in julProbList]
avgNovProbList = [-1 if len(x) == 0 else np.mean(x) for x in novProbList]

In [ ]:
fig=plt.figure(figsize=(12,8))
columns = 1
rows = 2

fig.add_subplot(rows, columns, 1)
fig.tight_layout()

plt.plot(list(range(24)), avgNoonProbList, color='r')
plt.plot(list(range(24)), avgEightAMProbList, color='g')
plt.plot(list(range(24)), avgEightPMProbList, color='b')

plt.xticks(list(range(24)))
plt.xlim((0, 23))
plt.ylim((-0.01, 1.01))

plt.ylabel("Consistency Probability")
plt.xlabel("Ground-truth Hour of Capture")
plt.grid(True, which='major', color='grey', linestyle='-.', linewidth=0.5, alpha=0.5)



fig.add_subplot(rows, columns, 2)
fig.tight_layout()

plt.plot(list(range(12)), avgJanProbList, color='r')
plt.plot(list(range(12)), avgJulProbList, color='g')
plt.plot(list(range(12)), avgNovProbList, color='b')


plt.xticks(list(range(12)), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.xlim((0, 11))
plt.ylim((-0.01, 1.01))

plt.ylabel("Consistency Probability")
plt.xlabel("Ground-truth Month of Capture")
plt.grid(True, which='major', color='grey', linestyle='-.', linewidth=0.5, alpha=0.5)


plt.show()